In [12]:
! pip install pandas


In [17]:
import os
import timeit
import multiprocessing as mp
import concurrent.futures
import numpy as np
import pandas as pd

In [2]:
import pandas as pd


In [20]:
df = pd.read_csv(r'C:\Users\Chaloh\Downloads\rfm_data.csv')

In [21]:
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])


In [60]:
today = df['PurchaseDate'].max()
rfm_data = df.groupby('CustomerID').agg({
    'PurchaseDate': lambda x: (today - x.max()).days,
    'TransactionAmount': 'sum',
    'OrderID': 'count'
}).reset_index()

In [61]:
rfm_data.columns = ['CustomerID', 'PurchaseDate', 'TransactionAmount', 'OrderID']

In [62]:
rfm_data['R_Score'] = pd.qcut(rfm_data['PurchaseDate'], q=4, labels=False)
rfm_data['F_Score'] = pd.qcut(rfm_data['TransactionAmount'], q=4, labels=False)
rfm_data['M_Score'] = pd.qcut(rfm_data['CustomerID'], q=4, labels=False)

In [63]:
rfm_data['RFM_Score'] = rfm_data['R_Score'].astype(str) + rfm_data['F_Score'].astype(str) + rfm_data['M_Score'].astype(str)

In [65]:
rfm_data['Segment'] = pd.cut(
    rfm_data['RFM_Score'].astype(int),
    bins=[0, 133, 233, 333, 444, 555],
    labels=['Need Attention', 'At Risk', 'Potential Loyalists', 'Loyal Customers', 'Champions']
)


In [66]:
print(rfm_data[['CustomerID', 'PurchaseDate','OrderID', 'TransactionAmount',  'RFM_Score', 'Segment']])
               

     CustomerID  PurchaseDate  OrderID  TransactionAmount RFM_Score  \
0          1011            33        2            1129.02       230   
1          1025            21        1             359.29       110   
2          1029             0        1             704.99       020   
3          1046            43        1             859.82       230   
4          1049            13        1             225.72       000   
..          ...           ...      ...                ...       ...   
941        9941            42        1             960.53       233   
942        9950            38        1             679.11       223   
943        9954            12        1             798.01       033   
944        9985            57        1              36.10       303   
945        9991            30        1             626.81       123   

                 Segment  
0                At Risk  
1         Need Attention  
2         Need Attention  
3                At Risk  
4           